In [21]:

import sys

sys.path.append("/gstore/project/paneldesign/pkg")

sys.path.append("/gstore/home/taol9/gitsss/yo/tao")
# import primer3
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
import csv
import pandas as pd
from collections import defaultdict,OrderedDict
from collections import Counter
import matplotlib.pyplot as plt

import pandas as pd
import os
import pickle
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
import csv
import pandas as pd
from collections import defaultdict,OrderedDict
from collections import Counter
import matplotlib.pyplot as plt
import csv
from Bio.Seq import Seq
import barcode

from myfunc import *
# from myfunc import primers2mips, rc


In [20]:
!pip install --target /gstore/project/paneldesign/pkg barcode 


  Using cached barcode-1.0.2-py3-none-any.whl
  Using cached python_Levenshtein-0.20.9-py3-none-any.whl (9.4 kB)
  Using cached Levenshtein-0.20.9-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (177 kB)
  Using cached rapidfuzz-2.11.1-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)


In [9]:
cd /home/jupyter/gitsss/notebooks/

[Errno 2] No such file or directory: '/home/jupyter/gitsss/notebooks/'
/gstore/home/taol9/gitsss/yo/oneamp


In [10]:
pwd

'/gstore/home/taol9/gitsss/yo/oneamp'

In [35]:
len(set(lig_barcodes))

250

# barcoded ligation

In [38]:
# Methy compitalbe ligation adaptors: 
ligation_strand='TAGTGGGATTCCTGCTGTCAGT'.replace("C","G")
lig_barcodes=bgen(10,maxstretch=1,distance=3)
lig_barcodes=[b for b in lig_barcodes if b[-1]!='T']  # rule out barcodes T is the last base
lig_barcodes=[b.replace('C','A') for b in lig_barcodes] # rule out C bases for all positions
lig_barcodes=list(set(lig_barcodes))
umi='WW'  # AT
rcumi='SS' # CG
candidates=dict()
for bar in lig_barcodes:
    tmp=dict()
    long_strand=ligation_strand+umi+bar
    short_strand="TA"+rc(bar)+rcumi
    tmp["long_strand"]=long_strand
    tmp["short_strand"]=short_strand
    candidates[bar]=tmp
    

In [ ]:
# !mkdir ../oneamp

In [42]:
ligation_strand

'TAGTGGGATTGGTGGTGTGAGT'

In [89]:
dftt=pd.DataFrame.from_dict(candidates,orient='index')
outname='../oneamp/oneamp_bar_lig_adaptor.xlsx'
dftt.to_excel(outname)

In [90]:
df = dftt.sample(n=24)
outname='../oneamp/oneamp_bar_lig_adaptor_24.xlsx'
df.to_excel(outname)

# pcr amp primer for stnd ampliwga kit, can be shared with bar lig libs

In [41]:
# ampli1_MseLig_21="AGTGGGATTCCGCATGCTAGC"

ampli_universal="TAGTGGGATTCCTGCTGTCAGT"
ampli1_kit_adaptor="AGTGGGATTCCTGCTGTCAGT"
HT_R1SP_w ='ACACTCTTTCCCTACACGACGCTCTTCCGATCT'
HT_R2SP_w ='AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC'

p5_w = "AATGATACGGCGACCACCGAGATCTACAC"
p7_w = 'ATCTCGTATGCCGTCTTCTGCTTG'


ME_w="AGATGTGTATAAGAGACAG"

NT_s5_w="TCGTCGGCAGCGTC"
NT_s7_c="GTCTCGTGGGCTCGG"

HT_R2SP_c =  rc(HT_R2SP_w) 
HT_R1SP_c =rc(HT_R1SP_w)

p7_c = rc(p7_w)
p5_c = rc(p5_w)

ME_w="AGATGTGTATAAGAGACAG"

NT_s5_w="TCGTCGGCAGCGTC"
NT_s7_c="GTCTCGTGGGCTCGG"
T7_w="TAATACGACTCACTATAG"

In [43]:
def bgen(length,maxstretch=3,distance=3,):
    candi=[]
    for i in range(30000):
        x=str(RandomDNA(length))
        if 45<GC(x)<55:
            candi.append(x)
    candi=barcode.filter_stretches(candi, maxstretch)
    candi=barcode.filter_distance(candi, distance)
    return candi

In [44]:
idxs=bgen(8,3)
len(idxs)

411

In [46]:
ampli_lib_primer=dict()
for i in range(96):
    tmp=dict()
    tmp['idx7']="lib7_"+str(idxs[i])
    tmp['idx5']='lib5_'+str(idxs[-i])
    AdaptorA=p7_c+idxs[i]+NT_s7_c+ligation_strand
    AdaptorB=p5_w+idxs[-i]+NT_s5_w+ligation_strand
    tmp['libprimer7']=str(AdaptorA)
    tmp['libprimer5']=str(AdaptorB)
    ampli_lib_primer[i]=tmp



In [47]:
dftt=pd.DataFrame.from_dict(ampli_lib_primer,orient='index')
outname='./oneamp_lib_primer.xlsx'
dftt.to_excel(outname)

# for oneamp

In [88]:
    read2=(NT_s7_c+ligation_strand)
    read1=(NT_s5_w+ligation_strand)
    index1=rc(read2)[-33:]
    index2=rc(read1)[-33:]
print ("oneamp_read1seqprimer,", read1[-30:])
print ("oneamp_read2seqprimer,", read2[-28:])
print ("oneamp_index1primer,", index1[-27:])
print ("oneamp_index2primer,", index2[-27:])

oneamp_read1seqprimer, GCAGCGTCTAGTGGGATTGGTGGTGTGAGT
oneamp_read2seqprimer, GCTCGGTAGTGGGATTGGTGGTGTGAGT
oneamp_index1primer, CCAATCCCACTACCGAGCCCACGAGAC
oneamp_index2primer, ACCAATCCCACTAGACGCTGCCGACGA


## for ampliwgs 

In [86]:
    read1=(NT_s5_w+ampli_universal+"TAA")
    read2=(NT_s7_c+ampli_universal+'TAA')
    index1=rc(read2)[-27:]
    index2=rc(read1)[-26:]
print ("ampli_bar_read1seqprimer,", read1[-34:])
print ("ampli_bar_read2seqprimer,", read2[-32:])
print ("ampli_bar_index1primer,", index1)
print ("ampli_bar_index2primer,", index2)    

ampli_bar_read1seqprimer, GGCAGCGTCTAGTGGGATTCCTGCTGTCAGTTAA
ampli_bar_read2seqprimer, GGCTCGGTAGTGGGATTCCTGCTGTCAGTTAA
ampli_bar_index1primer, GGAATCCCACTACCGAGCCCACGAGAC
ampli_bar_index2primer, GGAATCCCACTAGACGCTGCCGACGA
